In [1]:
import pandas as pd

In [1]:
from evaluate import generate_google_link, add_url_column

In [2]:
import pandas as pd 
test = pd.read_excel("C:\\Users\\Spencer\\Desktop\\clean\\data\\data.xlsx")
test.head()

,Company ID,Industry ID,Industry Name,Company Name,Address 1,City,State,Zip,Listing Type,Country,Phone,Google Link
0,2000626178,51,Locations: Chain Restaurants,Anthony's Coal Fired Pizza,984 State Rt 17,RAMSEY,NJ,07446-1607,Location,United States of America,2.018183e+09,https://www.google.com/maps/place/Anthony's+Co...
1,2000626186,51,Locations: Chain Restaurants,Anthony's Coal Fired Pizza,80 Parsonage Rd,EDISON,NJ,08837-2416,Location,United States of America,7.327442e+09,https://www.google.com/maps/place/Anthony's+Co...
2,2000626189,51,Locations: Chain Restaurants,Anthony's Coal Fired Pizza,6401 Jericho Tpke,COMMACK,NY,11725-2832,Location,United States of America,6.318583e+09,https://www.google.com/maps/place/Anthony's+Co...
3,2000626192,51,Locations: Chain Restaurants,Anthony's Coal Fired Pizza,50 East Wynnewood Road,WYNNEWOOD,PA,19096,Location,United States of America,6.106455e+09,https://www.google.com/maps/place/Anthony's+Co...
4,2000626194,51,Locations: Chain Restaurants,Anthony's Coal Fired Pizza,48 Walkers Brook Dr,READING,MA,01867-3224,Location,United States of America,7.819448e+09,https://www.google.com/maps/place/Anthony's+Co...


In [4]:
test.loc[:, ["Company Name", "Address 1", "City", "State", "Zip"]].apply(generate_google_link, axis=1)[0]

"https://www.google.com/search?q=Anthony's Coal Fired Pizza+984 State Rt 17+RAMSEY+NJ+07446-1607"

In [3]:
add_url_column(test, ["Company Name", "Address 1", "City", "State", "Zip"], "url")

In [1]:
from evaluate import evaluate_locations

In [2]:
data_path = "C:\\Users\\Spencer\\Desktop\\clean\\data\\scraps.csv"
output_path = "C:\\Users\\Spencer\\Desktop\\clean\\data\\review_counts.csv"
id_col = "Company ID"
search_terms = ["Company Name", "Address 1", "City", "State", "Zip"]
new_url_col = "url"

evaluate_locations(data_path=data_path, output_path=output_path, id_col=id_col, search_terms=search_terms, new_url_col=new_url_col)

160 Google reviews
132 Google reviews


WebDriverException: Message: chrome not reachable
  (Session info: chrome=86.0.4240.198)


In [2]:
import pandas as pd 
from utilities import get_files
files = get_files("C:\\Users\\Spencer\\Desktop\\clean\\data\\evaluations", "csv")

In [4]:
evaluations = pd.read_csv(files[0])
for file in files[1:]:
    evaluations = pd.concat([evaluations, pd.read_csv(file)], ignore_index=True)

In [5]:
evaluations.head()

,Store Company ID,reviews_scraped,difference,proportion,review_count
0,2000626213,293,-15.0,0.951299,308.0
1,2000626218,626,-16.0,0.975078,642.0
2,2000626221,889,-16.0,0.982320,905.0
3,2000626224,728,-13.0,0.982456,741.0
4,2000626226,333,-5.0,0.985207,338.0


In [6]:
len(evaluations)

16385

In [7]:
master = pd.read_csv("C:\\Users\\Spencer\\Desktop\\clean\\data\\data.csv")

In [36]:
scraped_master = master[master["Store Company ID"].isin(evaluations["Store Company ID"])]

In [45]:
diff = pd.read_csv("C:\\Users\\Spencer\\Desktop\\clean\\data\\diff.csv")

In [19]:
diff = diff.rename(columns={"Company ID":"Store Company ID"})

In [20]:
diff.to_csv("diff.csv", index=False)

In [59]:
evaluations[(evaluations["proportion"] < 0.9)]

,Store Company ID,reviews_scraped,difference,proportion,review_count
25,2000626208,80,-684.0,0.104712,764.0
39,2000747544,20,-340.0,0.055556,360.0
86,2000626635,880,-733.0,0.545567,1613.0
116,2000626735,640,-324.0,0.663900,964.0
147,2000626780,450,-130.0,0.775862,580.0
...,...,...,...,...,...
16227,2000198833,10,-63.0,0.136986,73.0
16241,1000889791,8,-1.0,0.888889,9.0
16314,2000060509,69,-8.0,0.896104,77.0
16360,2000632581,26,-54.0,0.325000,80.0


In [43]:
scraped_evals = pd.merge(scraped_master, evaluations, on=["Store Company ID"])

In [54]:
rc = pd.read_csv("C:\\Users\\Spencer\\Desktop\\clean\\data\\review_counts.csv")

In [60]:
diff_rc = pd.merge(diff, rc, on=["Store Company ID"])

In [62]:
diff_rc["difference"] = -diff_rc["review_count"]

In [71]:
master_rc = pd.concat([scraped_evals, diff_rc], ignore_index=True)

In [98]:
evals2 = evaluations.drop_duplicates()

In [108]:
eval_dupes = evals2[evals2["Store Company ID"].duplicated()]

In [109]:
evals2["Store Company ID"].value_counts()

2000334388    3
2000334383    3
2000303507    3
2000303518    2
2000303505    2
             ..
2000629327    1
1000813431    1
1000947277    1
2000588363    1
1000964096    1
Name: Store Company ID, Length: 16325, dtype: int64

In [122]:
evals_with_dupes = evals2[evals2["Store Company ID"].isin(eval_dupes["Store Company ID"])]

In [138]:
id_dict = {}
rows_to_drop = []
for row in evals_with_dupes.iterrows():
    company_id = row[1]["Store Company ID"]
    reviews_scraped = row[1]["reviews_scraped"]
    if company_id not in id_dict.keys():
        id_dict[company_id] = row
    else:
        old_reviews_scraped = id_dict[company_id][1]["reviews_scraped"]
        if (reviews_scraped <= old_reviews_scraped):
            rows_to_drop.append(row)
        else:
            rows_to_drop.append(id_dict[company_id])
            id_dict[company_id] = row
rows_to_drop

[(1117, Store Company ID    1.000847e+09
  reviews_scraped     1.000000e+01
  difference         -9.450000e+02
  proportion          1.047120e-02
  review_count        9.550000e+02
  Name: 1117, dtype: float64), (65, Store Company ID    2.000334e+09
  reviews_scraped     8.000000e+00
  difference          0.000000e+00
  proportion          1.000000e+00
  review_count        8.000000e+00
  Name: 65, dtype: float64), (66, Store Company ID    2.000304e+09
  reviews_scraped     4.800000e+01
  difference          5.000000e+00
  proportion          1.116279e+00
  review_count        4.300000e+01
  Name: 66, dtype: float64), (1613, Store Company ID    1.000855e+09
  reviews_scraped     1.030000e+02
  difference         -2.000000e+00
  proportion          9.809524e-01
  review_count        1.050000e+02
  Name: 1613, dtype: float64), (1635, Store Company ID    1.000970e+09
  reviews_scraped     5.900000e+01
  difference         -3.000000e+00
  proportion          9.516129e-01
  review_count    

In [137]:
d[1]["Store Company ID"]

2000303508.0

In [146]:
sum([x-1 for x in list(evals2["Store Company ID"].value_counts())])

23

In [147]:
len(rows_to_drop)

23

In [149]:
indices_to_drop = [row[0] for row in rows_to_drop]

In [170]:
list(evals2.iloc[indices_to_drop].index)

[1123,
 65,
 66,
 1619,
 1641,
 1656,
 1663,
 5300,
 1674,
 5302,
 1687,
 14552,
 14608,
 16080,
 16146,
 16147,
 16149,
 16150,
 16151,
 16152,
 16153,
 16079,
 16291]

In [172]:
evals2[~evals2.index.isin(list(evals2.iloc[indices_to_drop].index))]

,Store Company ID,reviews_scraped,difference,proportion,review_count
0,2000626213,293,-15.0,0.951299,308.0
1,2000626218,626,-16.0,0.975078,642.0
2,2000626221,889,-16.0,0.982320,905.0
3,2000626224,728,-13.0,0.982456,741.0
4,2000626226,333,-5.0,0.985207,338.0
...,...,...,...,...,...
16380,2000632587,212,-5.0,0.976959,217.0
16381,2000632589,281,-4.0,0.985965,285.0
16382,2000632595,89,-1.0,0.988889,90.0
16383,2000745550,10,0.0,1.000000,10.0


In [173]:
evals2.iloc[1117]

Store Company ID    1.000851e+09
reviews_scraped     3.330000e+02
difference          0.000000e+00
proportion          1.000000e+00
review_count        3.330000e+02
Name: 1123, dtype: float64

In [177]:
evals_nodupes = evals2.drop(indices_to_drop)

In [179]:
evals_nodupes[evals_nodupes["Store Company ID"].duplicated()]

,Store Company ID,reviews_scraped,difference,proportion,review_count


In [187]:
master_nodupes = pd.merge(scraped_master, evals_nodupes, on=["Store Company ID"])

In [191]:
master_rc_nodupes = pd.concat([master_nodupes, diff_rc], ignore_index=True)

In [192]:
master_rc_nodupes

,Store Company ID,Industry ID,Industry Name,Company Name,Address 1,City,State,Zip,Listing Type,Country,Phone,Google Link,reviews_scraped,difference,proportion,review_count
0,2000626178,51,Locations: Chain Restaurants,Anthony's Coal Fired Pizza,984 State Rt 17,RAMSEY,NJ,07446-1607,Location,United States of America,2.018183e+09,https://www.google.com/maps/place/Anthony's+Co...,385,-10.0,0.974684,395.0
1,2000626186,51,Locations: Chain Restaurants,Anthony's Coal Fired Pizza,80 Parsonage Rd,EDISON,NJ,08837-2416,Location,United States of America,7.327442e+09,https://www.google.com/maps/place/Anthony's+Co...,799,-25.0,0.969660,824.0
2,2000626189,51,Locations: Chain Restaurants,Anthony's Coal Fired Pizza,6401 Jericho Tpke,COMMACK,NY,11725-2832,Location,United States of America,6.318583e+09,https://www.google.com/maps/place/Anthony's+Co...,388,-10.0,0.974874,398.0
3,2000626192,51,Locations: Chain Restaurants,Anthony's Coal Fired Pizza,50 East Wynnewood Road,WYNNEWOOD,PA,19096,Location,United States of America,6.106455e+09,https://www.google.com/maps/place/Anthony's+Co...,322,-10.0,0.969880,332.0
4,2000626194,51,Locations: Chain Restaurants,Anthony's Coal Fired Pizza,48 Walkers Brook Dr,READING,MA,01867-3224,Location,United States of America,7.819448e+09,https://www.google.com/maps/place/Anthony's+Co...,203,-9.0,0.957547,212.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29018,2000743484,51,Locations: Chain Restaurants,VOCELLI PIZZA,5025 LINDBERG ST,LINCOLN,NE,68516,Location,United States of America,NaN,NaN,0,NaN,0.000000,NaN
29019,2000479267,51,Locations: Chain Restaurants,Vocelli Pizza,617 Warrendale Rd,GIBSONIA,PA,15044-8044,Location,United States of America,NaN,NaN,0,-38.0,0.000000,38.0
29020,2000479270,51,Locations: Chain Restaurants,Vocelli Pizza,1605 Jefferson Davis Hwy,FREDERICKSBURG,VA,22401-4651,Location,United States of America,5.403694e+09,https://www.google.com/maps/place/Vocelli+Pizz...,0,NaN,0.000000,NaN
29021,2000745554,51,Locations: Chain Restaurants,zpizza,650 LINCOLN,COLUMBIA,SC,29201,Location,United States of America,8.038887e+09,https://www.google.com/maps/place/650+Lincoln/...,0,-3.0,0.000000,3.0


In [195]:
master_rc_nodupes.to_csv("master_review_counts.csv")

In [9]:
rev_counts = pd.read_csv("C:\\Users\\Spencer\\Desktop\\clean\\data\\master_review_counts.csv")

In [6]:
from utilities import process_company_names

In [26]:
process_company_names(rev_counts, "Company Name", "company_name")

In [23]:
rev_counts = rev_counts.drop("Unnamed: 0", axis=1)

In [27]:
rev_counts.to_csv("master_review_counts.csv", index=False)

In [32]:
scraped_companies = rev_counts[rev_counts["reviews_scraped"]!=0]["company_name"].unique()

In [35]:
unscraped_company_locations = rev_counts[rev_counts["reviews_scraped"]==0][~rev_counts["company_name"].isin(scraped_companies)]

In [36]:
unscraped_company_locations["company_name"].unique()

array(['blackjack_pizza',
       'bostons_the_gourmet_pizza_restaurant_&_sports_bar', 'cobalt',
       'dominos', 'hungry_howies_pizza_&_subs', 'little_caesars',
       'little_caesars_pizza', 'pizza_italia', 'schmizza_pub_&_grub',
       'south_philly_steaks_&_fries', 'the_market', 'the_market_by_villa'],
      dtype=object)

In [39]:
final_unscraped_company_locations = unscraped_company_locations[~unscraped_company_locations["company_name"].isin(["dominos", "little_caesars", "little_caesars_pizza"])]

In [41]:
final_unscraped_company_locations.to_csv("C:\\Users\\Spencer\\Desktop\\clean\\data\\unscraped_company_locations.csv", index=False)

In [43]:
unsearchable_locations = final_unscraped_company_locations[final_unscraped_company_locations["review_count"].isnull()]

In [46]:
unsearchable_locations["company_name"].unique()

array(['cobalt', 'hungry_howies_pizza_&_subs', 'pizza_italia',
       'schmizza_pub_&_grub', 'south_philly_steaks_&_fries', 'the_market'],
      dtype=object)

In [47]:
searchable_locations = final_unscraped_company_locations[~final_unscraped_company_locations["review_count"].isnull()]

In [48]:
searchable_locations["company_name"].unique()

array(['blackjack_pizza',
       'bostons_the_gourmet_pizza_restaurant_&_sports_bar',
       'the_market_by_villa'], dtype=object)

In [52]:
searchable_locations_final = searchable_locations[searchable_locations["company_name"]!="the_market_by_villa"]

In [53]:
searchable_locations_final.to_csv("unscraped_searchable.csv", index=False)

In [57]:
unsearchable_locations_final = unsearchable_locations[~unsearchable_locations["company_name"].isin(["pizza_italia"])]

In [58]:
unsearchable_locations_final["company_name"].unique()

array(['cobalt', 'hungry_howies_pizza_&_subs', 'schmizza_pub_&_grub',
       'south_philly_steaks_&_fries', 'the_market'], dtype=object)

In [59]:
searchable_locations_final["company_name"].unique()

array(['blackjack_pizza',
       'bostons_the_gourmet_pizza_restaurant_&_sports_bar'], dtype=object)

In [60]:
unsearchable_locations_final.to_csv("unscraped_unsearchable.csv", index=False)

In [1]:
from evaluate import evaluate_all

In [ ]:
data_path = "C:\\Users\\Spencer\\Desktop\\clean\\data\\scraps.csv"
output_path = "C:\\Users\\Spencer\\Desktop\\clean\\data\\review_counts_closed_2019.csv"
id_col = "Company ID"
search_terms = ["Company Name", "Address 1", "City", "State", "Zip"]
new_url_col = "url"

evaluate_locations(data_path=data_path, output_path=output_path, id_col=id_col, search_terms=search_terms, new_url_col=new_url_col)